In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
np.random.seed(1232)
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\dep')
data = pd.read_csv("Rand_train.csv", sep=',')
data_val = pd.read_csv("Rand_valid.csv", sep=',')

In [3]:
data.cost = (np.exp(data.cost) - 0.5)*((data.cost != 0) * 1)
data_val.cost = (np.exp(data_val.cost) - 0.5)*((data_val.cost != 0) * 1)
data = shuffle(data).reset_index(drop=True)
data = data.drop(['fmde', 'lxghindx', 'lpi'], axis=1)
data_val = data_val.drop(['fmde', 'lxghindx', 'lpi'], axis=1)

In [4]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

ranking = np.load('ranking.npy')

my_index = X.columns.values[ranking]
X2 = X[my_index]
X2_val = X_val[my_index]

In [5]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [6]:
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\prog_calc')
from fit5_nn import fit, comb_error

In [11]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
        self.fc13 = nn.Linear(2, 1, bias = True)
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):        
               
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2)) 
        x2 = self.fc23(x2)
        
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)
        x1 = torch.cat((x2, x1), 1)
        x1 = self.fc13(x1)
        
        
        x1 = torch.sigmoid(x1)        
        x2 = torch.exp(x2)        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [23]:
n_epochs = 5000
lr = 0.001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 10
clipping = 0.20
sig2 = 950000
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining3.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [26]:
net = Net1()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

#PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining3.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, 
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

Epoch 1: Training Loss: 1.407216211160024, Validation Loss: 1.5275300741195679
Epoch 2: Training Loss: 1.4050328930219014, Validation Loss: 1.524045467376709
Epoch 3: Training Loss: 1.4012312094370525, Validation Loss: 1.5192017555236816
Epoch 4: Training Loss: 1.3962731957435608, Validation Loss: 1.5133793354034424
Epoch 5: Training Loss: 1.3904789686203003, Validation Loss: 1.5068535804748535
Epoch 6: Training Loss: 1.3840564489364624, Validation Loss: 1.4998313188552856
Epoch 7: Training Loss: 1.3772225379943848, Validation Loss: 1.492473840713501
Epoch 8: Training Loss: 1.3701329429944356, Validation Loss: 1.4848840236663818
Epoch 9: Training Loss: 1.362717628479004, Validation Loss: 1.4771411418914795
Epoch 10: Training Loss: 1.355260153611501, Validation Loss: 1.4692929983139038
Epoch 11: Training Loss: 1.3477085828781128, Validation Loss: 1.461388349533081
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 1.340121825536092, Validation Loss: 1.4534558057785034
Epoch 13: Tra

Epoch 100: Training Loss: 0.9003296792507172, Validation Loss: 1.0083166360855103
Epoch 101: Training Loss: 0.8996168076992035, Validation Loss: 1.007657527923584
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.8989280958970388, Validation Loss: 1.0070054531097412
Epoch 103: Training Loss: 0.898249626159668, Validation Loss: 1.0063456296920776
Epoch 104: Training Loss: 0.8975973625977834, Validation Loss: 1.0056849718093872
Epoch 105: Training Loss: 0.8969345092773438, Validation Loss: 1.0050065517425537
Epoch 106: Training Loss: 0.8962688545385996, Validation Loss: 1.0043046474456787
Epoch 107: Training Loss: 0.8955848515033722, Validation Loss: 1.003585696220398
Epoch 108: Training Loss: 0.8948946495850881, Validation Loss: 1.0028560161590576
Epoch 109: Training Loss: 0.8941989143689474, Validation Loss: 1.0021190643310547
Epoch 110: Training Loss: 0.8934828738371531, Validation Loss: 1.0013813972473145
Epoch 111: Training Loss: 0.8927914798259735, Validation Loss: 1.00063

Epoch 197: Training Loss: 0.8407858113447825, Validation Loss: 0.9394707679748535
Epoch 198: Training Loss: 0.8401214877764384, Validation Loss: 0.9382292032241821
Epoch 199: Training Loss: 0.8394173781077067, Validation Loss: 0.936950147151947
Epoch 200: Training Loss: 0.8386477331320444, Validation Loss: 0.9357223510742188
Epoch 201: Training Loss: 0.8379553059736887, Validation Loss: 0.9344220757484436
New Checkpoint Saved into PATH
Epoch 202: Training Loss: 0.8371590177218119, Validation Loss: 0.9330950975418091
Epoch 203: Training Loss: 0.836421271165212, Validation Loss: 0.9316372275352478
Epoch 204: Training Loss: 0.8356217940648397, Validation Loss: 0.9300423264503479
Epoch 205: Training Loss: 0.8347155153751373, Validation Loss: 0.928324818611145
Epoch 206: Training Loss: 0.8337835768858591, Validation Loss: 0.9264981746673584
Epoch 207: Training Loss: 0.8328167895476023, Validation Loss: 0.924567699432373
Epoch 208: Training Loss: 0.8318422933419546, Validation Loss: 0.922535

Epoch 293: Training Loss: 0.8076146841049194, Validation Loss: 0.8719920516014099
Epoch 294: Training Loss: 0.8075580696264902, Validation Loss: 0.8718701004981995
Epoch 295: Training Loss: 0.8075665235519409, Validation Loss: 0.8714096546173096
Epoch 296: Training Loss: 0.8074991703033447, Validation Loss: 0.8711416125297546
Epoch 297: Training Loss: 0.807479610045751, Validation Loss: 0.8712117671966553
Epoch 298: Training Loss: 0.8074558476607004, Validation Loss: 0.8715220093727112
Epoch 299: Training Loss: 0.807356725136439, Validation Loss: 0.8716126680374146
Epoch 300: Training Loss: 0.8073626458644867, Validation Loss: 0.8722420334815979
Epoch 301: Training Loss: 0.8073065479596456, Validation Loss: 0.8721619844436646
New Checkpoint Saved into PATH
Epoch 302: Training Loss: 0.8072535792986552, Validation Loss: 0.8720887899398804
Epoch 303: Training Loss: 0.8072663446267446, Validation Loss: 0.8725523948669434
Epoch 304: Training Loss: 0.8072365323702494, Validation Loss: 0.8730

Epoch 390: Training Loss: 0.8046027421951294, Validation Loss: 0.870943009853363
Epoch 391: Training Loss: 0.8045384089152018, Validation Loss: 0.8705828785896301
Epoch 392: Training Loss: 0.8044767379760742, Validation Loss: 0.8701518774032593
Epoch 393: Training Loss: 0.8045102953910828, Validation Loss: 0.869461715221405
Epoch 394: Training Loss: 0.8044854899247488, Validation Loss: 0.8692258596420288
Epoch 395: Training Loss: 0.8045715689659119, Validation Loss: 0.8689906001091003
Epoch 396: Training Loss: 0.8045400381088257, Validation Loss: 0.8689371347427368
Epoch 397: Training Loss: 0.8044818639755249, Validation Loss: 0.8692325949668884
Epoch 398: Training Loss: 0.8044906655947367, Validation Loss: 0.8698968291282654
Epoch 399: Training Loss: 0.8043281237284342, Validation Loss: 0.8700993657112122
Epoch 400: Training Loss: 0.8043230473995209, Validation Loss: 0.8703562021255493
Epoch 401: Training Loss: 0.8043059209982554, Validation Loss: 0.8706558346748352
New Checkpoint Sav

...

Epoch 4548: Training Loss: 0.7802232603232065, Validation Loss: 0.8569457530975342
Epoch 4549: Training Loss: 0.7802366316318512, Validation Loss: 0.8571847081184387
Epoch 4550: Training Loss: 0.7802110215028127, Validation Loss: 0.8572834134101868
Epoch 4551: Training Loss: 0.780193010965983, Validation Loss: 0.8574820756912231
Epoch 4552: Training Loss: 0.780203104019165, Validation Loss: 0.8574191331863403
Epoch 4553: Training Loss: 0.7802337805430094, Validation Loss: 0.8577187061309814
Epoch 4554: Training Loss: 0.7802300155162811, Validation Loss: 0.8574076294898987
Epoch 4555: Training Loss: 0.7802093426386515, Validation Loss: 0.857563316822052
Epoch 4556: Training Loss: 0.7802286843458811, Validation Loss: 0.8577150106430054
Epoch 4557: Training Loss: 0.7802175084749857, Validation Loss: 0.8576193451881409
Epoch 4558: Training Loss: 0.7802077929178873, Validation Loss: 0.857259213924408
Epoch 4559: Training Loss: 0.7801839808622996, Validation Loss: 0.8573665022850037
Epoch 45

Epoch 4649: Training Loss: 0.7799941202004751, Validation Loss: 0.8563703298568726
Epoch 4650: Training Loss: 0.7800373633702596, Validation Loss: 0.8562250137329102
Epoch 4651: Training Loss: 0.7800321280956268, Validation Loss: 0.856474757194519
Epoch 4652: Training Loss: 0.7800039748350779, Validation Loss: 0.8567923903465271
Epoch 4653: Training Loss: 0.7800454795360565, Validation Loss: 0.8572468161582947
Epoch 4654: Training Loss: 0.7799790898958842, Validation Loss: 0.8577063083648682
Epoch 4655: Training Loss: 0.7800552348295847, Validation Loss: 0.8585887551307678
Epoch 4656: Training Loss: 0.7800337274869283, Validation Loss: 0.8585780262947083
Epoch 4657: Training Loss: 0.7800552596648535, Validation Loss: 0.858640193939209
Epoch 4658: Training Loss: 0.7800283133983612, Validation Loss: 0.858329176902771
Epoch 4659: Training Loss: 0.7799550195535024, Validation Loss: 0.8580805063247681
Epoch 4660: Training Loss: 0.7799256344636282, Validation Loss: 0.857536792755127
Epoch 46

Epoch 4749: Training Loss: 0.7797053754329681, Validation Loss: 0.8574070930480957
Epoch 4750: Training Loss: 0.779857317606608, Validation Loss: 0.858100950717926
Epoch 4751: Training Loss: 0.7798588077227274, Validation Loss: 0.8584901094436646
Epoch 4752: Training Loss: 0.7798839708169302, Validation Loss: 0.8588802218437195
Epoch 4753: Training Loss: 0.7798745135466257, Validation Loss: 0.8589908480644226
Epoch 4754: Training Loss: 0.7798247635364532, Validation Loss: 0.858473002910614
Epoch 4755: Training Loss: 0.7796881397565206, Validation Loss: 0.858062744140625
Epoch 4756: Training Loss: 0.7796858052412668, Validation Loss: 0.8576650023460388
Epoch 4757: Training Loss: 0.7798049648602804, Validation Loss: 0.8570501208305359
Epoch 4758: Training Loss: 0.779710034529368, Validation Loss: 0.8566815853118896
Epoch 4759: Training Loss: 0.7799408237139384, Validation Loss: 0.8561189770698547
Epoch 4760: Training Loss: 0.7798838416735331, Validation Loss: 0.8560988306999207
Epoch 476

Epoch 4848: Training Loss: 0.7794108390808105, Validation Loss: 0.8574162125587463
Epoch 4849: Training Loss: 0.779401550690333, Validation Loss: 0.8579704761505127
Epoch 4850: Training Loss: 0.7794093191623688, Validation Loss: 0.8581972122192383
Epoch 4851: Training Loss: 0.7795134683450063, Validation Loss: 0.858695387840271
Epoch 4852: Training Loss: 0.7794841130574545, Validation Loss: 0.8590490818023682
Epoch 4853: Training Loss: 0.7796236972014109, Validation Loss: 0.8593744039535522
Epoch 4854: Training Loss: 0.7795277734597524, Validation Loss: 0.8588650822639465
Epoch 4855: Training Loss: 0.7794169187545776, Validation Loss: 0.8582229018211365
Epoch 4856: Training Loss: 0.7794595758120219, Validation Loss: 0.8573964834213257
Epoch 4857: Training Loss: 0.7794682780901591, Validation Loss: 0.8570228815078735
Epoch 4858: Training Loss: 0.7794660727183024, Validation Loss: 0.8574419617652893
Epoch 4859: Training Loss: 0.7793958783149719, Validation Loss: 0.8577088117599487
Epoch 

Epoch 4947: Training Loss: 0.7793579498926798, Validation Loss: 0.8570268750190735
Epoch 4948: Training Loss: 0.7792665759722391, Validation Loss: 0.8567805290222168
Epoch 4949: Training Loss: 0.779304176568985, Validation Loss: 0.8565409779548645
Epoch 4950: Training Loss: 0.7793287734190623, Validation Loss: 0.8565264344215393
Epoch 4951: Training Loss: 0.7792507211367289, Validation Loss: 0.8568545579910278
Epoch 4952: Training Loss: 0.7793877323468527, Validation Loss: 0.8579527139663696
Epoch 4953: Training Loss: 0.7793122033278147, Validation Loss: 0.8585037589073181
Epoch 4954: Training Loss: 0.7793154915173849, Validation Loss: 0.8582919836044312
Epoch 4955: Training Loss: 0.7793130179246267, Validation Loss: 0.8582581281661987
Epoch 4956: Training Loss: 0.7793218096097311, Validation Loss: 0.8583862781524658
Epoch 4957: Training Loss: 0.7792956531047821, Validation Loss: 0.8585571646690369
Epoch 4958: Training Loss: 0.7793796509504318, Validation Loss: 0.8587379455566406
Epoch

In [27]:
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining3.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [28]:
with torch.no_grad():
    val_inputs = torch.FloatTensor(X2_val.values)
    val_labels = torch.FloatTensor(Y2_val)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
    val_outputs = net.forward(val_inputs)
    val_loss = m_error(val_outputs, val_labels) 
    print(val_loss.item())

0.8577356934547424


In [29]:
val_outputs

tensor([[5.3541e-01, 5.1972e+01],
        [2.8562e-01, 1.0496e+02],
        [4.7210e-01, 5.6025e+01],
        ...,
        [2.5011e-01, 1.5122e+02],
        [3.2343e-02, 4.4642e+02],
        [7.0091e-01, 1.5632e+00]])

In [30]:
val_labels

tensor([[  1.0000,  -0.0000],
        [  1.0000,  -0.0000],
        [  1.0000,  -0.0000],
        ...,
        [  0.0000,  10.5326],
        [  0.0000, 113.7847],
        [  1.0000,  -0.0000]])

In [ ]:
val_outputs = val_outputs.numpy()
val_labels = val_labels.numpy()
y_scores = val_outputs[:,0]
y_real = val_labels[:,0]

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    plt.figure(figsize=(8,8))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.legend(loc='best')

In [ ]:
fpr, tpr, auc_thresholds = roc_curve(y_real, y_scores)
print(auc(fpr, tpr)) # AUC of ROC
plot_roc_curve(fpr, tpr, 'recall_optimized')

In [ ]:
y = val_labels[:,1][val_labels[:,1]>0]
yhat = val_outputs[:,1][val_labels[:,1]>0]

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
y = np.array(Y_val)  #so we can tune the cut point
p = val_outputs[:,0]
f2 = val_outputs[:,1]
yhat = (1-p)*f2

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
print(np.cov(y, p))

In [ ]:
print(np.cov(f2, p))